In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part'):
    shutil.rmtree('./file/2.part')

os.makedirs('./file/2.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.url/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = input_.loc[a, 'Url']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            title = html.xpath('//head/title/text()')[0].strip()
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                type_ = html.xpath('//span[@itemprop="name"]')[-1].xpath('./text()')[0].strip()

                # = = = = = = = = = = = = = = =

                tag = html.xpath('//p[@class="product-title"]/text()')[0].strip() if html.xpath('//p[@class="product-title"]/text()') else ''

                # = = = = = = = = = = = = = = =

                dict_src, list_src = {}, html.xpath('//div[@class="oneaCarousel onea-carousel-container onea-carousel-vertical"]/ul[@class="oneaCarouselBody onea-carousel-body invisible-scroll-y"]/li/img[@class="img-sq-s lazy-img lazyOnReady carouselDisplaySelect"]/@data-replacement')
                for i, src in enumerate(list_src):
                    dict_src[str(i)] = f'''https:{src.strip()}'''
                
                # = = = = = = = = = = = = = = =

                list_div, dict_count = html.xpath('//div[@id="product-description-tab"]/div/ul/li'), {}
                for div in list_div:
                    list_text = [text.strip().removesuffix(':').strip() for text in div.xpath('./span/text()')]
                    if len(list_text) > 1:
                        if list_text[0] in dict_count:
                            dict_count[list_text[0]] += 1
                        else:
                            dict_count[list_text[0]] = 1
                
                list_specification = html.xpath('//div[@id="product-description-tab"]/specifications/specification')
                for specification in list_specification:
                    key = specification.xpath('./text()')[0].strip()
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                        dict_count[key] = 1
                
                list_p = html.xpath('//div[@id="product-description-tab"]/p')
                for p in list_p:
                    if p.xpath('./strong/text()'):
                        key = p.xpath('./strong/text()')[0].strip().removesuffix(':').strip()
                        if key in ['Item Condition', 'WARNING']:
                            if key in dict_count:
                                dict_count[key] += 1
                            else:
                                dict_count[key] = 1
            
                # = = = = = = = = = = = = = = =

                i, dict_description, dict_no = 0, {}, {}
                for div in list_div:
                    list_text = [text.strip().removesuffix(':').strip() for text in div.xpath('./span/text()')]
                    if len(list_text) == 2:
                        if dict_count[list_text[0]] == 1:
                            dict_description[str(i)] = {list_text[0]: list_text[1]}
                        else:
                            if list_text[0] in dict_no:
                                dict_no[list_text[0]] += 1
                            else:
                                dict_no[list_text[0]] = 1
                            dict_description[str(i)] = {f'{list_text[0]}-{dict_no[list_text[0]]}': list_text[1]}
                    elif len(list_text) > 2:
                        if dict_count[list_text[0]] == 1:
                            dict_description[str(i)] = {list_text[0]: {}}
                            for j, text in enumerate(list_text[1:]):
                                dict_description[str(i)][list_text[0]][str(j)] = text
                        else:
                            if list_text[0] in dict_no:
                                dict_no[list_text[0]] += 1
                            else:
                                dict_no[list_text[0]] = 1
                            dict_description[str(i)] = {f'{list_text[0]}-{dict_no[list_text[0]]}': {}}
                            for j, text in enumerate(list_text[1:]):
                                dict_description[str(i)][f'{list_text[0]}-{dict_no[list_text[0]]}'][str(j)] = text
                    i += 1
                
                for j, specification in enumerate(list_specification):
                    key = specification.xpath('./text()')[0].strip()
                    if dict_count[key] == 1:
                        dict_description[str(i)] = {key: html.xpath('//div[@id="product-description-tab"]/specifications/value')[j].xpath('./text()')[0].strip()}
                    else:
                        if key in dict_no:
                            dict_no[key] += 1
                        else:
                            dict_no[key] = 1
                        dict_description[str(i)] = {f'{key}-{dict_no[key]}': html.xpath('//div[@id="product-description-tab"]/specifications/value')[j].xpath('./text()')[0].strip()}
                    i += 1
                
                for p in list_p:
                    if p.xpath('./strong/text()'):
                        key = p.xpath('./strong/text()')[0].strip().removesuffix(':').strip()
                        if key == 'Item Condition':
                            if dict_count[key] == 1:
                                dict_description[str(i)] = {key: p.xpath('./span/text()')[0].strip()}
                            else:
                                if key in dict_no:
                                    dict_no[key] += 1
                                else:
                                    dict_no[key] = 1
                                dict_description[str(i)] = {f'{key}-{dict_no[key]}': p.xpath('./span/text()')[0].strip()}
                            i += 1
                        elif key == 'WARNING':
                            if dict_count[key] == 1:
                                dict_description[str(i)] = {key: p.xpath('./text()')[0].strip()}
                            else:
                                if key in dict_no:
                                    dict_no[key] += 1
                                else:
                                    dict_no[key] = 1
                                dict_description[str(i)] = {f'{key}-{dict_no[key]}': p.xpath('./text()')[0].strip()}
                            i += 1
            
                # = = = = = = = = = = = = = = =

                dict_ = json.loads(resp.text.split('<!-- Google structured data  Markup -->')[1].split('<script type="application/ld+json">')[1].split('</script>')[0])

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Type': type_,
                                         'SKU': dict_['mpn'].strip() if dict_['mpn'] else '',
                                         'Part Number': dict_['sku'].strip(),
                                         'Brand': dict_['brand']['name'].strip(),
                                         'Tag': tag,
                                         'Title': title,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'OEM Url': f'''https://www.1aauto.com/catalog/product/fit?skuId={resp.text.split('window.productId =')[1].split(';')[0].strip()}&pathName={input_.loc[a, 'Url'].removeprefix('https://www.1aauto.com').replace('/', '%2F')}''',
                                         'Json_Description': json.dumps(dict_description),
                                         'Price': f'''{dict_['offers']['priceCurrency'].strip()} {dict_['offers']['price']}''',
                                         'Rating': dict_['aggregateRating']['ratingValue'].strip() if dict_['aggregateRating'] else '',
                                         'Rating Count': int(dict_['aggregateRating']['ratingCount'].strip()) if dict_['aggregateRating'] else 0}])
                                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part/{file}',
              f'''./file/2.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：3

[状态：ok，尝试次数：1] - 100.00% - crawler_2-part_error > 862.https://www.1aauto.com/ford-front-driver-side-door-weatherstrip-seal-fairchild-automotive-f3004/i/1awsd00176
[剩余数量：0] - [当前时间：08:34:32]

[状态：ok，尝试次数：1] - 100.00% - crawler_2-part_error > 1246.https://www.1aauto.com/chevrolet-gmc-olds-inner-and-outer-window-sweep-4-piece-set-fairchild-automotive-kg2006/i/1awss00115
[剩余数量：0] - [当前时间：08:34:33]

[状态：ok，尝试次数：1] - 100.00% - crawler_2-part_error > 660.https://www.1aauto.com/1978-81-chevrolet-pontiac-roofrail-weatherstrip-seal-2-piece-set-metro-moulded-parts-rr-5020/i/1awsr00095
[剩余数量：0] - [当前时间：08:34:35]

输出ing...



Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 2941.31it/s]

Done ~
